In [1]:
# import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup

import re

import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

# there are 36 states+UT in India
df_bs.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,,\nNorth America\n,"2,180,855","+5,164","131,990",+847,"867,722","+2,265","1,181,143","19,471",,,,,,North America,\n,,
1,,\nSouth America\n,"1,030,697",+607,"46,090",+15,"489,396","+5,034","495,211","11,695",,,,,,South America,\n,,
2,,\nEurope\n,"2,054,517","+9,359","177,262",+162,"1,051,525","+9,435","825,730","9,305",,,,,,Europe,\n,,
3,,\nAsia\n,"1,266,505","+6,504","33,144",+86,"767,886","+2,398","465,475","14,529",,,,,,Asia,\n,,
4,,\nAfrica\n,"172,183",,"4,783",,"75,830",,"91,570",449,,,,,,Africa,\n,,
5,,\nOceania\n,"8,857",+7,124,,"8,266",+33,467,4,,,,,,Australia/Oceania,\n,,
6,,\n\n,721,,15,,651,,55,4,,,,,,,\n,,
7,,World,"6,714,335","+21,641","393,408","+1,110","3,261,276","+19,165","3,059,651","55,457",861,50.5,,,,All,\n,,
8,1,USA,"1,924,189",+138,"110,179",+6,"712,252",,"1,101,758","17,083","5,816",333,"19,568,069","59,142","330,864,650",North America,172,"3,003",17
9,2,Brazil,"615,870",,"34,039",,"274,997",,"306,834","8,318","2,899",160,"986,365","4,643","212,451,006",South America,345,"6,241",215


In [6]:
df_bs.tail()

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
212,205,Seychelles,11,,,,11,,0,,112,,,,"98,304",Africa,"8,937",,
213,206,MS Zaandam,9,,2,,,,7,,,,,,,,,,
214,207,Western Sahara,9,,1,,6,,2,,15,2,,,"596,147",Africa,"66,239","596,147",
215,208,British Virgin Islands,8,,1,,7,,0,,265,33,167,"5,527","30,217",North America,"3,777","30,217",181
216,209,Papua New Guinea,8,,,,8,,0,,0.9,,"4,646",520,"8,933,791",Australia/Oceania,"1,116,724",,"1,923"


In [7]:
df_bs = df_bs.loc[8:].reset_index(drop=True)
df_bs = df_bs.drop('#', axis=1)
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,USA,"1,924,189",+138,"110,179",+6,"712,252",,"1,101,758","17,083","5,816",333,"19,568,069","59,142","330,864,650",North America,172,"3,003",17
1,Brazil,"615,870",,"34,039",,"274,997",,"306,834","8,318","2,899",160,"986,365","4,643","212,451,006",South America,345,"6,241",215
2,Russia,"449,834","+8,726","5,528",+144,"212,680","+8,057","231,626","2,300","3,083",38,"12,053,663","82,599","145,930,189",Europe,324,"26,398",12
3,Spain,"287,740",,"27,133",,N/A,N/A,N/A,617,"6,154",580,"4,063,843","86,921","46,753,542",Europe,162,"1,723",12
4,UK,"281,661",,"39,904",,N/A,N/A,N/A,604,"4,151",588,"5,005,565","73,762","67,860,771",Europe,241,"1,701",14


In [8]:
df_bs = df_bs.iloc[:, :-3]

In [9]:
df_bs.columns

Index(['Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/\n1M pop\n',
       'Population', 'Continent'],
      dtype='object')

In [10]:
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"330,864,650","1,924,189",+138,"110,179",+6,"712,252",,"1,101,758","17,083","5,816",333,"19,568,069","59,142"
1,Brazil,South America,"212,451,006","615,870",,"34,039",,"274,997",,"306,834","8,318","2,899",160,"986,365","4,643"
2,Russia,Europe,"145,930,189","449,834","+8,726","5,528",+144,"212,680","+8,057","231,626","2,300","3,083",38,"12,053,663","82,599"
3,Spain,Europe,"46,753,542","287,740",,"27,133",,N/A,N/A,N/A,617,"6,154",580,"4,063,843","86,921"
4,UK,Europe,"67,860,771","281,661",,"39,904",,N/A,N/A,N/A,604,"4,151",588,"5,005,565","73,762"


In [11]:
df_bs.columns[2:]

Index(['Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop'],
      dtype='object')

In [12]:
for col in df_bs.columns[2:]:
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)
 

df_bs.replace('', np.nan)

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,330864650,1924189,138,110179,6,712252,NaN,1101758,17083,5816,333,19568069,59142
1,Brazil,South America,212451006,615870,NaN,34039,NaN,274997,NaN,306834,8318,2899,160,986365,4643
2,Russia,Europe,145930189,449834,8726,5528,144,212680,8057,231626,2300,3083,38,12053663,82599
3,Spain,Europe,46753542,287740,NaN,27133,NaN,NaN,NaN,NaN,617,6154,580,4063843,86921
4,UK,Europe,67860771,281661,NaN,39904,NaN,NaN,NaN,NaN,604,4151,588,5005565,73762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Seychelles,Africa,98304,11,NaN,NaN,NaN,11,NaN,0,NaN,112,NaN,NaN,NaN
205,MS Zaandam,NaN,NaN,9,NaN,2,NaN,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN
206,Western Sahara,Africa,596147,9,NaN,1,NaN,6,NaN,2,NaN,15,2,NaN,NaN
207,British Virgin Islands,North America,30217,8,NaN,1,NaN,7,NaN,0,NaN,265,33,167,5527


In [13]:
df_bs.to_csv('worldometer_data.csv', index=False)